In [49]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

#MODEL = "gemini-pro" # Chat model, returns AIMessage object
MODEL = "llama2" # Completion model, returns string 
#MODEL = "llama3"

In [50]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gemini"):
    model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model=MODEL)
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings()
model.invoke("Tell me a joke")


'Why was the math book sad? Because it had too many problems! 😂'

In [51]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser # Pipeline where model output passed as parser input
chain.invoke("Tell me a joke")

'Why was the math book sad? Because it had too many problems! 😂'

In [52]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)


[Document(page_content='Building Machine Learning Systems That Don\'tSuckA live, interactive program that\'ll help you build production-readymachinelearning systems from the ground up.Next cohort:\xa0July1 - 18, 2024Check the schedulefor more details about upcoming cohorts.\nI want to join!Sign inLearn how to design, build, deploy, and scalemachine learningsystems to solve real-world problems.I\'ll lose my mind if I see another book or course teaching people the same basicideas for the hundredth time. Most people are stuck in beginner mode, and findinghelp to solve real-world problems is hard.I want to change that.I started writing software 30 years ago. I\'ve written pipelines and trained modelsfor some of the largest companies in the world. I want to show you how to do the"This is the best machine learning course I\'ve done.Worth every cent."— Jose Reyes, AI/ML at Cevo AustraliaBuilding Machine Learning Systems That Don\'t Suckhttps://www.ml.school/\n1 of 1102/06/24, 9:16 pm', metada

In [53]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [54]:
chain = prompt | model | parser

In [55]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [56]:
chain.invoke(
    {
        "context": "Socrates is a man. All men are mortal.",
        "question": "Is Socrates mortal?" 
    }
)

'Of course! Here is my answer based on the context you provided:\n\nYes, Socrates is mortal. Since all men are mortal and Socrates is a man, it follows that he is also mortal.'

In [57]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [61]:
# Retriever is a langchain component used to retrieve info from anywhere
retriever = vectorstore.as_retriever()
retriever.invoke("Machine Learning")

[Document(page_content="Real-life examples and case studiesLearn from practical experience building machinelearning systems that work in the real world.\nLive, interactive sessionsAsk questions and interact with the instructor andother students in real time.\nCode walkthroughsStep by step coding instructions to help you builda production system from scratch.Upcoming Schedule\nBuilding Machine Learning Systems That Don't Suckhttps://www.ml.school/\n5 of 1102/06/24, 9:16 pm", metadata={'source': 'mlschool.pdf', 'page': 4}),
 Document(page_content='• Hyperparameter tuning and experiment tracking.• Measuring the quality of your holdout set.• An introduction to distributed training using data parallelism and modelparallelism.See the remaining sessions\n"This is an awesome course! This is my second round and I continue learning. I recommend itwith complete confidence."\nJuan OlanoMachine Learning EngineerFrequently Asked QuestionsIf you can\'t find the answer to your question, please reach o

In [62]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt 
    | model 
    | parser)

chain.invoke({"question": "What is machine learning?"})

" Based on the context provided, machine learning appears to be a practical and hands-on program for technical professionals who are ready to put in the work to learn how to solve real-world problems using machine learning. The program covers various topics such as hyperparameter tuning, experiment tracking, distributed training, and more.\n\nFrom the context, it can be inferred that machine learning is a field of study focused on building machine learning systems that don't suck, which means the program is designed to help professionals build practical and reliable machine learning systems. The program also covers real-life examples and case studies, live, interactive sessions with the instructor and other students, and code walkthroughs to help learners build a production system from scratch.\n\nTherefore, based on the context provided, machine learning appears to be a field of study that involves building practical and reliable machine learning systems using various techniques and t

In [ ]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()